In [1]:
import cv2
import time
import numpy as np
import HandTrackingModule as htm
import math
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [2]:
# wCam, hCam = 640, 480

# cap = cv2.VideoCapture(0)
# cap.set(3, wCam)
# cap.set(4, hCam)
# pTime = 0

# detector = htm.HandDetector()

# devices = AudioUtilities.GetSpeakers()
# interface = devices.Activate(
#     IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
# volume = interface.QueryInterface(IAudioEndpointVolume)
# volumeRange = volume.GetVolumeRange()
# minimumVolume = volumeRange[0]
# maximumVolume = volumeRange[1]

# while True:
#     success, img = cap.read()
#     img = cv2.flip(img, 1)  # Flipping is needed to create mirroring
#     # Find the hands
#     detector.findHands(img)
#     lmList = detector.findPosition(img, draw=False)
#     if len(lmList) != 0:
#         # Filter based on size

#         # Find the distance btwn index and thumb

#         # Converting length to volume

#         # Reduce resolution to make smoother.

#         # Check fingers which are up

#         # if pinky is down then set volume

#         # drawings

#         # Frame rate

#         x1, y1 = lmList[4][1], lmList[4][2]
#         x2, y2 = lmList[8][1], lmList[8][2]
#         cx, cy = (x1+x2)//2, (y1+y2)//2
#         cv2.circle(img, (x1, y1), 8, (255, 0, 255), cv2.FILLED)
#         cv2.circle(img, (x2, y2), 8, (255, 0, 255), cv2.FILLED)
#         cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 2)
#         cv2.circle(img, (cx, cy), 8, (255, 0, 255), cv2.FILLED)

#         length = math.hypot(x2-x1, y2-y1)
#         # Hand range 10 to 150
#         # Volume range -65.25 to 0
#         vol = np.interp(length, [10, 150], [minimumVolume, maximumVolume])
#         volume.SetMasterVolumeLevel(vol, None)

#     cTime = time.time()
#     fps = 1/(cTime-pTime)
#     pTime = cTime

#     cv2.putText(img, f'FPS:{int(fps)}', (48, 70),
#                 cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 3)
#     cv2.imshow("Image", img)
#     cv2.waitKey(1)

In [3]:
wCam, hCam = 640, 480
# boundingBox=[]
cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)
pTime = 0


detector = htm.HandDetector(maxHands=1)

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)
volumeRange = volume.GetVolumeRange()
minimumVolume = volumeRange[0]
maximumVolume = volumeRange[1]
area = 0
while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)  # Flipping is needed to create mirroring
    # Find the hands
    detector.findHands(img)
    lmList, boundingBox = detector.findPosition(img, handNo=0, draw=True)
    if len(lmList) != 0:
        # Filter based on size
        area = (boundingBox[2]-boundingBox[0]) * \
            (boundingBox[3]-boundingBox[1])//100
        # print(area)
        if 100 < area < 900:

            # Find the distance btwn index and thumb
            length, img, lineInfo = detector.findDistance(4, 8, img=img)

            # Converting length to volume
            vol = np.interp(length, [10, 150], [0, 100])

            # Reduce resolution to make smoother.
            smoothness = 5
            vol = smoothness*round(vol/smoothness)

            # Check fingers which are up
            fingers=detector.fingersUp()
          

            # if pinky is down then set volume
            if not fingers[4]:
                volume.SetMasterVolumeLevelScalar(vol/100, None)

            # drawings

            # Frame rate

            

    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime

    cv2.putText(img, f'FPS:{int(fps)}', (48, 70),
                cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 3)
    cv2.imshow("Image", img)
    cv2.waitKey(1)

KeyboardInterrupt: 